In [1]:

import pandas as pd 
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.model_selection import StratifiedKFold,  GridSearchCV, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import gc


# Обучение модели с CatBoost на данных с методом трансформации классов

In [2]:
df= pd.read_csv(r'X5_clients_data_train.csv', sep = ';')
med = df.age.median()
df.age = df.age.apply(lambda x: x if((x >= 16) and (x <= 90)) else med)
df.age = df.age.fillna(med)
df.Mean_diffs = df.Mean_diffs.fillna(df.Mean_diffs.median()) 
df = df.fillna(0)
df.describe()
display(df)

,client_id,regular_points_received,express_points_received,regular_points_spent,express_points_spent,regular_balance,express_balance,Total_Amount,Amount_BonusDiscount_purchs,BonusDiscount,...,PArt_CTM_purchs,cnt_alc_purchs,cnt_CTM_purchs,Mean_diffs,first_order_date,age,LoveBonuses,Gender,treatment_flg,target
0,27159c9807,17.1,0.0,0.0,0.0,17.1,0.0,2950.17,0.00,0.000000,...,0.000000,0,0,10.0,2018-11-26 13:09:43,26.0,0,-1,0,1
1,b1b5ea0726,21.2,0.0,6.0,0.0,15.2,0.0,2334.64,190.75,0.030496,...,0.800000,0,4,19.0,2018-12-22 10:20:58,35.0,0,-1,1,0
2,8399c72ebf,26.2,0.0,0.0,0.0,26.2,0.0,4534.55,0.00,0.000000,...,0.444444,0,8,6.0,2018-11-25 09:49:15,23.0,2,0,1,0
3,d5b8aff1ad,13.7,0.0,0.0,0.0,13.7,0.0,2359.99,0.00,0.000000,...,0.538462,0,7,5.0,2019-01-15 09:59:17,69.0,0,-1,1,1
4,a6fd598f71,90.4,0.0,149.0,0.0,-58.6,0.0,8259.32,1014.49,0.128063,...,0.647059,0,11,6.0,2018-11-27 09:50:48,23.0,0,-1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200034,fecd01be95,6.4,0.0,0.0,0.0,6.4,0.0,1398.80,0.00,0.000000,...,0.555556,0,5,13.0,2018-11-28 18:36:33,73.0,0,0,1,0
200035,929c68106b,12.9,0.0,0.0,0.0,12.9,0.0,2051.93,0.00,0.000000,...,1.000000,2,4,34.0,2018-11-30 16:56:31,69.0,0,0,0,1
200036,f183f1a613,19.9,0.0,36.0,40.0,-16.1,-40.0,4271.41,426.61,0.151211,...,0.307692,3,4,6.0,2018-12-18 11:48:22,43.0,0,-1,0,1
200037,226757e95c,12.1,0.0,0.0,0.0,12.1,0.0,2737.92,0.00,0.000000,...,0.769231,1,10,8.0,2018-11-26 15:11:55,29.0,0,0,1,0


In [3]:
df = df.drop(['MeanItemCost', 'client_id'
                , 'Mean_TOP_1_BRAND'
                ,	'Mean_TOP_2_BRAND'
                ,	'Mean_TOP_3_BRAND'
                ,	'Mean_TOP_4_BRAND',
                	'Mean_TOP_5_BRAND',
                    'Mean_TOP_1_SEG',
                    'Mean_TOP_2_SEG',
                    'Mean_TOP_3_SEG',
                    'Mean_TOP_4_SEG',
                    'Mean_TOP_5_SEG',
                    'first_order_date'	], axis = 1)
# X = df.iloc[:, 0:-1].set_index('client_id')
# y = df.iloc[:, [0, -1]].set_index('client_id')
# del df
# gc.collect()
treat = df.treatment_flg.values
targ = df.target.values
treat

array([0, 1, 1, ..., 0, 1, 0], dtype=int64)

In [4]:

p = df[df.treatment_flg == 1].shape[0] / df.shape[0]
X = df.iloc[:, 0:-2].values
X = (X - X.mean()) / X.std()
y = df.target * (df.treatment_flg - p) / (1- p) / p
y = y.values
Y = np.array(list(zip(y, df.treatment_flg.values)))
del df
gc.collect()
print(X)
print(y)
print(p)
# X_train, X_test, y_train, y_test = train_test_split(X, y)
# output.enable_custom_widget_manager()
# grid = {'learning_rate':  np.linspace(0, 0.05, 10)[1:-1],
#         'depth': [4, 6, 10],
#         'l2_leaf_reg': [1, 3, 5, 7, 9]}
# clf = CatBoostClassifier(verbose = 0)  
# grid_search_result = clf.grid_search(grid, 
#                                        X=X_train, 
#                                        y=y_train, 
#                                        plot=True)
from sklift.metrics import (
    uplift_at_k, uplift_auc_score, qini_auc_score, weighted_average_uplift
)

from sklift.viz import plot_qini_curve, plot_uplift_curve, plot_uplift_by_percentile
import matplotlib.pyplot as plt


[[-0.14500764 -0.15234497 -0.15234497 ... -0.14118879 -0.15234497
  -0.15277405]
 [-0.14324839 -0.15234497 -0.14977047 ... -0.13732704 -0.15234497
  -0.15277405]
 [-0.14110297 -0.15234497 -0.15234497 ... -0.14247604 -0.1514868
  -0.15234497]
 ...
 [-0.1438062  -0.15234497 -0.13689795 ... -0.13389437 -0.15234497
  -0.15277405]
 [-0.14715306 -0.15234497 -0.15234497 ... -0.13990154 -0.15234497
  -0.15234497]
 [-0.12085022 -0.15234497 -0.15234497 ... -0.14161787 -0.15234497
  -0.15277405]]
[-1.99923045  0.          0.         ... -1.99923045  0.
 -1.99923045]
0.4998075375301816


In [5]:
Y

array([[-1.99923045,  0.        ],
       [ 0.        ,  1.        ],
       [ 0.        ,  1.        ],
       ...,
       [-1.99923045,  0.        ],
       [ 0.        ,  1.        ],
       [-1.99923045,  0.        ]])

In [9]:

X_train,X_test,y_tr,y_ts= train_test_split(X, Y)

In [10]:
y_train = y_tr.T[0]
treat_train = y_tr.T[1]

y_test = y_ts.T[0]
treat_test = y_ts.T[1]

In [12]:
rf = CatBoostRegressor(verbose = 0)# Instantiate the grid search model
#rf.fit(X_train, y_train)

In [13]:


grid = {
    'max_depth': [1, 2, 4, 6, 10, 20],
    'learning_rate': [0.03, 0.1, 0.02, 0.05],
    'min_data_in_leaf': [1, 2, 3, 4, 6, 10],
    'n_estimators': [10, 50, 100, 1000]
}# Create a based model


grid_search_result = rf.grid_search(grid, 
                                       X=X_train, 
                                       y=y_train, 
                                       cv = 3,
                                       plot=True)


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))


bestTest = 1.580644328
bestIteration = 9

0:	loss: 1.5806443	best: 1.5806443 (0)	total: 406ms	remaining: 3m 53s

bestTest = 1.58051214
bestIteration = 9

1:	loss: 1.5805121	best: 1.5805121 (1)	total: 484ms	remaining: 2m 18s

bestTest = 1.580681719
bestIteration = 9

2:	loss: 1.5806817	best: 1.5805121 (1)	total: 558ms	remaining: 1m 46s

bestTest = 1.580582717
bestIteration = 9

3:	loss: 1.5805827	best: 1.5805121 (1)	total: 653ms	remaining: 1m 33s

bestTest = 1.580466723
bestIteration = 49

4:	loss: 1.5804667	best: 1.5804667 (4)	total: 982ms	remaining: 1m 52s

bestTest = 1.580436355
bestIteration = 31

5:	loss: 1.5804364	best: 1.5804364 (5)	total: 1.36s	remaining: 2m 9s

bestTest = 1.580499692
bestIteration = 49

6:	loss: 1.5804997	best: 1.5804364 (5)	total: 1.72s	remaining: 2m 19s

bestTest = 1.5804465
bestIteration = 48

7:	loss: 1.5804465	best: 1.5804364 (5)	total: 2.03s	remaining: 2m 24s

bestTest = 1.580452397
bestIteration = 87

8:	loss: 1.5804524	best: 1.5804364 (5)	total: 2.74s	

In [ ]:
grid_search_result

{'params': {'min_data_in_leaf': 1,
  'depth': 2,
  'iterations': 100,
  'learning_rate': 0.1},
 'cv_results': defaultdict(list,
             {'iterations': [0,
               1,
               2,
               3,
               4,
               5,
               6,
               7,
               8,
               9,
               10,
               11,
               12,
               13,
               14,
               15,
               16,
               17,
               18,
               19,
               20,
               21,
               22,
               23,
               24,
               25,
               26,
               27,
               28,
               29,
               30,
               31,
               32,
               33,
               34,
               35,
               36,
               37,
               38,
               39,
               40,
               41,
               42,
               43,
               44,
             

In [ ]:
grid_search_result['params']

In [ ]:
best_rf  = CatBoostRegressor(params = grid_search_result['params'])
best_rf.fit(X_train, y_train)

{'min_data_in_leaf': 1, 'depth': 2, 'iterations': 100, 'learning_rate': 0.1}

In [ ]:
y_pred = best_rf.predict(X_test)

In [ ]:
from sklift.metrics import (
    uplift_at_k, uplift_auc_score, qini_auc_score, weighted_average_uplift
)

from sklift.viz import plot_qini_curve, plot_uplift_curve, plot_uplift_by_percentile
import matplotlib.pyplot as plt

wau = weighted_average_uplift(y_true=y_test , uplift=y_pred,
                              treatment=treat_test)
uplift_overall = uplift_at_k(y_true=y_test , uplift=y_pred,
                              treatment= treat_test, strategy='by_group', k=0.3)

print(f'WAU = {wau}; UpLift 30% = {uplift_overall}')
fig, ax = plt.subplots(1, 2, figsize = (30, 10))
ax[0].set_title('Qini Curve')
ax[1].set_title('UpLift Curve')
plot_qini_curve(y_true=y_test , uplift=y_pred,
                        treatment=treat_test, ax = ax[0], perfect=False)
gc.collect()
plot_uplift_curve(y_true=y_test , uplift=y_pred,
                        treatment=treat_test, ax = ax[1], perfect=False)
gc.collect()
gc.collect()
gc.collect()
plt.savefig(f"Plots/Catboost_best_tune.png")

plot_uplift_by_percentile(y_true=y_test, uplift=y_pred,
                        treatment=treat_test, kind='bar')
plt.savefig(f"Plots/Catboost_best_tune_Procentile.png")

In [ ]:
# # Метод двух моделей
# #Default data
# skf = KFold(n_splits=5)
# skf.get_n_splits(X, y)
# metrics = {'wau':[], 'uplift_k':[]}
# i = 0

# for train_index, test_index in skf.split(X, y):
#       gc.collect()
#       X_train_, X_test_ = X[train_index], X[test_index]
#       y_train_, y_test_ = y[train_index], y[test_index]
#       reg = CatBoostRegressor(verbose=0)
#       reg.fit(X_train_, y_train_)
#       y_pred = reg.predict(X_test_)
#       del reg
#       del X_train_
#       del y_train_
#       del X_test_
#       gc.collect()
#       wau = weighted_average_uplift(y_true=targ[test_index] , uplift=y_pred,
#                               treatment=treat[test_index])
#       uplift_overall = uplift_at_k(y_true=targ[test_index] , uplift=y_pred,
#                               treatment=treat[test_index], strategy='by_group', k=0.3)
#       metrics['wau'].append(wau)
#       metrics['uplift_k'].append(uplift_overall)
#       i += 1
#       print(f'Split {i}; WAU = {wau}; UpLift 30% = {uplift_overall}')
#       fig, ax = plt.subplots(1, 2, figsize = (30, 10))
#       ax[0].set_title('Qini Curve')
#       ax[1].set_title('UpLift Curve')
#       plot_qini_curve(y_true=targ[test_index] , uplift=y_pred,
#                               treatment=treat[test_index], ax = ax[0], perfect=False)
#       gc.collect()
#       plot_uplift_curve(y_true=targ[test_index] , uplift=y_pred,
#                               treatment=treat[test_index], ax = ax[1], perfect=False)
#       gc.collect()
#       gc.collect()
#       gc.collect()
#       plt.savefig(f"Plots/Regression model Split {i}.png")

#       plot_uplift_by_percentile(y_true=targ[test_index], uplift=y_pred,
#                               treatment=treat[test_index], kind='bar')
#       plt.savefig(f"Plots/Regression model Procentile Split {i}.png")


# wau = np.round(np.mean(metrics['wau']), 4)
# uplift_overall = np.round(np.mean(metrics['uplift_k']), 4)
# print(f'Mean WAU = {wau}; Mean UpLift 30% = {uplift_overall}')


In [ ]:
gc.collect()

71372